<a href="https://colab.research.google.com/github/Yamini-Ranganathan/Predictive-ML-model-for-Myocardial-Infarction-morbidity/blob/main/Bonus_1_NER_tner_%26_BERT_Yamini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tuning on NER with T-NER
Ref:https://github.com/asahi417/tner

In [ ]:
%pip install pip -U
%pip install sentencepiece
%pip install sortedcontainers==2.1.0

In [ ]:
# main package
%pip install tner

In [ ]:
!head -n 5 train.txt

Super	B-Malware
Mario	I-Malware
Run	I-Malware
Malware	I-Malware
#	O


In [ ]:
from tner import TrainTransformersNER
trainer = TrainTransformersNER(
        checkpoint_dir='./ckpt_additional_training',
        dataset='/content/custom_data',
        transformers_model='albert-base-v1',  # the smallest bert only 11M parameters
        random_seed=1234,
        lr=1e-5,
        total_step=100,
        warmup_step=10,
        batch_size=64,
        max_seq_length=128)

2022-04-19 17:37:55 INFO     *** initialize network ***
2022-04-19 17:37:55 INFO     create new checkpoint
2022-04-19 17:37:55 INFO     checkpoint: ./ckpt_additional_training_e95663c2f57d40827f9ba8b15c863776
2022-04-19 17:37:55 INFO      - [arg] dataset: /content/custom_data
2022-04-19 17:37:55 INFO      - [arg] transformers_model: albert-base-v1
2022-04-19 17:37:55 INFO      - [arg] random_seed: 1234
2022-04-19 17:37:56 INFO      - [arg] lr: 1e-05
2022-04-19 17:37:56 INFO      - [arg] total_step: 100
2022-04-19 17:37:56 INFO      - [arg] warmup_step: 10
2022-04-19 17:37:56 INFO      - [arg] weight_decay: 1e-07
2022-04-19 17:37:56 INFO      - [arg] batch_size: 64
2022-04-19 17:37:56 INFO      - [arg] max_seq_length: 128
2022-04-19 17:37:56 INFO      - [arg] fp16: False
2022-04-19 17:37:56 INFO      - [arg] max_grad_norm: 1.0
2022-04-19 17:37:56 INFO      - [arg] lower_case: False


In [ ]:
trainer.train()

2022-04-19 17:38:01 INFO     target dataset: ['/content/custom_data']
2022-04-19 17:38:01 INFO     data_name: /content/custom_data
2022-04-19 17:38:01 INFO     formatting custom dataset from /content/custom_data
2022-04-19 17:38:01 INFO     found following files: {'train': 'train.txt', 'test': 'test.txt', 'valid': 'valid.txt'}
2022-04-19 17:38:01 INFO     note that files should be named as either `valid.txt`, `test.txt`, or `train.txt` 
2022-04-19 17:38:01 INFO     dataset /content/custom_data/train.txt: 2810 entries
2022-04-19 17:38:01 INFO     dataset /content/custom_data/test.txt: 747 entries
2022-04-19 17:38:01 INFO     dataset /content/custom_data/valid.txt: 812 entries
Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertForTokenClassification: ['predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias']
- Th

In [ ]:
trainer.test(test_dataset='test.txt', batch_size_validation=1)

2022-04-19 17:42:20 INFO     testing model on test.txt
2022-04-19 17:42:20 INFO     data_loader: ['test', 'valid']
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2022-04-19 17:42:39 INFO     [epoch 2] (test) 
                precision    recall  f1-score   support

    Indicator       0.43      0.32      0.36       290
      Malware       0.17      0.00      0.01       241
 Organization       0.00      0.00      0.00       134
       System       0.40      0.01      0.02       245
Vulnerability       0.00      0.00      0.00        10

    micro avg       0.42      0.10      0.17       920
    macro avg       0.20      0.07      0.08       920
 weighted avg       0.29      0.10      0.12       920

2022-04-19 17:43:00 INFO     [epoch 2] (val

In [ ]:
import pandas as pd
metrics_df = pd.read_json('/content/ckpt_additional_training_e95663c2f57d40827f9ba8b15c863776/test_test.txt.json', lines=True)
print(metrics_df)

                                                test  \
0  {'f1': 16.57940663176265, 'recall': 10.3260869...   

                                               valid  
0  {'f1': 22.626262626262626, 'recall': 14.395886...  


Fine tuning NER with BERT

Ref: https://medium.com/@andrewmarmon/fine-tuned-named-entity-recognition-with-hugging-face-bert-d51d4cb3d7b5

https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb


In [1]:
!pip install transformers datasets seqeval

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 325 kB 53.3 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 6.6 MB 38.1 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 895 kB 53.2 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 1.1 MB 43.5 MB/s 
     |████████████████████████████████| 212 kB 68.6 MB/s 
     |████████████████████████████████| 136 kB 68.0 MB/s 
     |████████████████████████████████| 127 kB 48.5 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 57.6 MB/s 
     |████████████████████████████████| 271 kB 54.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=67512b9d2fe15fc052a7e9041efc30e69c69bc1b292d06a6646f313b874c675b
  Stored in directory: /root/.cache/pip/whe

In [2]:
import os
import itertools
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from datasets import Dataset
from datasets import load_metric
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification


In [3]:
from pathlib import Path
import re

def read_file(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)
    return token_docs, tag_docs

In [19]:
texts, tags = read_file('train.txt')
test_token, test_label = read_file('test.txt')
val_token, val_label = read_file('valid.txt')

train_df= pd.read_csv('train.txt', sep="\s+", names = ['tokens', 'ner_tags'], header = None)
test_df= pd.read_csv('test.txt', sep="\s+", names = ['tokens', 'ner_tags'], header = None)
val_df = pd.read_csv('valid.txt', sep="\s+", names = ['tokens', 'ner_tags'], header = None)
train_df

,tokens,ner_tags
0,Super,B-Malware
1,Mario,I-Malware
2,Run,I-Malware
3,Malware,I-Malware
4,#,O
...,...,...
26833,grow,O
26834,in,O
26835,the,O
26836,future,O


In [21]:
custom_labels = list(train_df['ner_tags'].unique())
print(custom_labels)

labels_to_ids = {k: v for v, k in enumerate(train_df.ner_tags.unique())}
ids_to_labels = {v: k for v, k in enumerate(train_df.ner_tags.unique())}
labels_to_ids

['B-Malware', 'I-Malware', 'O', 'B-System', 'I-System', 'B-Organization', 'B-Indicator', 'I-Organization', 'I-Indicator', 'B-Vulnerability', 'I-Vulnerability']


{'B-Indicator': 6,
 'B-Malware': 0,
 'B-Organization': 5,
 'B-System': 3,
 'B-Vulnerability': 9,
 'I-Indicator': 8,
 'I-Malware': 1,
 'I-Organization': 7,
 'I-System': 4,
 'I-Vulnerability': 10,
 'O': 2}

In [36]:
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        tokens = [[x.split('\t')[0] for x in y] for y in split_list]
        entities = [[x.split('\t')[1][:-1] for x in y] for y in split_list] 
    return pd.DataFrame({'tokens': tokens, 'ner_tags': entities})

In [37]:
train_df = get_tokens_and_ner_tags('train.txt')
test_df = get_tokens_and_ner_tags('test.txt')
val_df = get_tokens_and_ner_tags('valid.txt')

In [38]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

In [41]:
task = "ner" 
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
label_list = custom_labels
label_encoding_dict = labels_to_ids

In [40]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [44]:

def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [45]:
trainData = train_dataset.map(tokenize_and_align_labels, batched=True)
testData = test_dataset.map(tokenize_and_align_labels, batched=True)
valData = val_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [46]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels_to_ids))

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

In [50]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)


In [51]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [52]:
from datasets import load_dataset, load_metric
metric = load_metric("seqeval")

In [53]:
labels = labels_to_ids.keys()
metric.compute(predictions=[labels], references=[labels])

{'Indicator': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'Malware': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'Organization': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'System': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'Vulnerability': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [54]:
import numpy as np

idx2tag = dict([(value, key) for key, value in labels_to_ids.items()])
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [idx2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [idx2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [55]:
training_args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=trainData,
    eval_dataset=valData,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [56]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2811
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 176


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.258985,0.719045,0.701467,0.710147,0.925786


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 813
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=176, training_loss=0.45812368392944336, metrics={'train_runtime': 1742.4884, 'train_samples_per_second': 1.613, 'train_steps_per_second': 0.101, 'total_flos': 78399001892436.0, 'train_loss': 0.45812368392944336, 'epoch': 1.0})

In [57]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 813
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 1.0,
 'eval_accuracy': 0.9257855039728956,
 'eval_f1': 0.7101470374144708,
 'eval_loss': 0.25898516178131104,
 'eval_precision': 0.7190448113207547,
 'eval_recall': 0.7014667817083693,
 'eval_runtime': 103.0682,
 'eval_samples_per_second': 7.888,
 'eval_steps_per_second': 0.495}

In [59]:
predictions, labels, _ = trainer.predict(testData)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 748
  Batch size = 16


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Indicator': {'f1': 0.7260374586852735,
  'number': 2330,
  'precision': 0.6344672657252889,
  'recall': 0.8484978540772532},
 'Malware': {'f1': 0.43470149253731344,
  'number': 605,
  'precision': 0.4989293361884368,
  'recall': 0.3851239669421488},
 'Organization': {'f1': 0.0, 'number': 241, 'precision': 0.0, 'recall': 0.0},
 'System': {'f1': 0.16837782340862423,
  'number': 399,
  'precision': 0.4659090909090909,
  'recall': 0.10275689223057644},
 'Vulnerability': {'f1': 0.0, 'number': 47, 'precision': 0.0, 'recall': 0.0},
 'overall_accuracy': 0.8986299168307285,
 'overall_f1': 0.6173042643630879,
 'overall_precision': 0.6131844184146009,
 'overall_recall': 0.6214798453892877}